## Scrapper Automatization

In [2]:
#import library
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd


In [3]:
#resquests website
section_url='https://www.ultimahora.com/contenidos/nacional.html'
resquests_section_url= requests.get(section_url)

In [4]:
#Verify the website is online
resquests_section_url.status_code


200

In [ ]:
#extracts the elements of the news

def get_info(url):
    dict={}
    r_notice= requests.get(url)
    s_notice= BeautifulSoup(r_notice.text, 'lxml')

    #title notice
    title=s_notice.find('h1', attrs={'class':'title'}).text
    if title:
        dict['title']=title
    else:
        dict['title']=None

    #date notice
    time=s_notice.find('time', attrs={'class':'date'}).text
    if time:
        dict['time']=time
    else:
        dict['time']=None

    #lead
    lead=s_notice.find('h2', attrs={'class':'excerpt'}).text
    if time:
        dict['lead']=lead
    else:
        dict['lead']=None
    
    #body
    b=''
    body=s_notice.find('div', attrs={'class':'body-content note-body'}).find_all('p')
    for p in body:
        b=b+''+(p.text)
    if b:
        dict['body']=b
    else:
        dict['body']=None

    #img
    media = s_notice.find('div', attrs={'class':'image'}).find('img')
    img_src=media.get('data-td-src-property')
    try:
        r_img=requests.get(img_src)
        if r_img.status_code==200:
            dict['img']=r_img.content
        else:
            dict['img']=None
    except:
        print('there is not image')

    return dict




In [ ]:
get_info('https://www.ultimahora.com/corte-rechaza-accion-contra-ordenanza-que-limita-distancia-gasolineras-asuncion-n3040554.html')

In [ ]:
def scraper_url(url):
    try:
        r=requests.get(url)
    except Exception as e:
        print('Error ',url)
        print (e)
        return None
    
    if r.status_code !=200:
        print(f'Error status code {url}')
        print(e)
        return None
    
    dict=get_info(url)
    dict['url']=url

    return dict

In [ ]:
scraper_url('https://www.ultimahora.com/corte-rechaza-accion-contra-ordenanza-que-limita-distancia-gasolineras-asuncion-n3040554.html')

In [ ]:
def get_next_page_url(url):
    list_next_page_url=[url]

    for i in range(5):
        r= requests.get(list_next_page_url[-1])
        s= BeautifulSoup(r.text, 'lxml')
        get_next_page= s.find('a', attrs={'class':'right-button'})
        get_url=get_next_page.get('href')
        list_next_page_url.append(get_url)
    
    return list_next_page_url

In [ ]:
get_next_page_url('https://www.ultimahora.com/contenidos/nacional.html')

In [ ]:
#Extract the links of the notices
def get_notice_url(url):
    all_notice_url=[]
    for i in url:
        try:
            r=requests.get(i)
            s=BeautifulSoup(r.text, 'lxml')
            xpath_section_url= etree.HTML(str(s))
            links_notices=xpath_section_url.xpath("//div[@class='col-12 col-lg-8']//*[@class='article-title']//a")
            
            #Save links in a list
            for i in links_notices:
                link=i.get("href")
                all_notice_url.append(link)
                
        except Exception as e:
            print ('Error \n')
            print(e)
    
    return all_notice_url

In [ ]:
get_notice_url(get_next_page_url('https://www.ultimahora.com/contenidos/nacional.html'))

In [ ]:
#scrapper
data=[]
all_URLs=get_notice_url(get_next_page_url(section_url))
for contador, url in enumerate(all_URLs):
    print (f'scraper link: {contador}/{len(all_URLs)}')
    data.append(scraper_url(url))

In [ ]:
len(data)

In [ ]:
# create data frame
df = pd.DataFrame(data)
df

In [ ]:
# create an excel document with the obtained data
df.to_excel('scraper.xlsx')